In [1]:
import pandas as pd

embedding = "te3l" # / "te3s"
grobid_model = "full_model"
no_prev_chunking = True

path = f"../data/dfs/{embedding}{'_no_prev_chunking' if no_prev_chunking else ''}/{grobid_model}/ReferenceErrorDetection_data_with_chunk_info.pkl"
print(path)

# read the dataframe from a pickle file
df = pd.read_pickle(path)

../data/dfs/te3l_no_prev_chunking/full_model/ReferenceErrorDetection_data_with_chunk_info.pkl


## Create the prompts

In [2]:
def format_excerpts(excerpt_list):
    excerpts_text = ""
    for id, excerpt in enumerate(excerpt_list):
        excerpts_text += f"Excerpt {id+1}: \n{excerpt}\n\n"
    return excerpts_text

In [3]:
print(format_excerpts(df.iloc[0]['Top_3_Chunk_Texts']))

Excerpt 1: 
Automatic implementation of fuzzy reasoning spiking neural P systems for diagnosing faults in complex power systems
		
			HNRong
		
		
			KYi
		
		
			GXZhang
		
		
			JDong
		
		
			PPaul
		
		
			ZHuang
		
	
	
		Complexity
		
			2019
			16
			2019
		
	



	
		A rough set-based bioinspired fault diagnosis method for electrical substations
		
			TWang
		
		
			WLiu
		
		
			JBZhao
		
	
	
		International Journal of Electrical Power & Energy Systems
		
			119
			105961
			2020
		
	



	
		Simplified and yet turing universal spiking neural P systems with polarizations optimized by anti-spikes
		
			TWu
		
		
			TZhang
		
		
			FXu
		
	
	
		Neurocomputing
		
			414
			
			2020

Excerpt 2: 
Nomenclature
ES:Expert system ANN:Artificial neural network PN:Petri net SNPS: Spiking neural P system rMFRSNPSs: Modified fuzzy reasoning spiking neural P systems with real numbers PVRA:Pulse value reasoning algorithm FFPRA:Forward fault prediction reasoning algorithm BAFDRA:Backward abducti

In [4]:
def create_prompt(df_row, corrected_statements=False):
    title = df_row['Citing Article Title']

    statement = df_row['Statement with Citation']
    statement = df_row["Statement with Citation"]
    if corrected_statements:
        statement = df_row["Corrected Statement"]
        if pd.isna(statement) or statement == "null":
            statement = df_row["Statement with Citation"]

    reference_title = df_row['Reference Article Title']
    reference_abstract = df_row['Reference Article Abstract']
    reference_excerpts = format_excerpts(df_row['Top_3_Chunk_Texts'])

    prompt = f"""   
You are an experienced scientific writer and editor. 
You will be given a statement from an article that cites a reference article and information from the reference article. 
You will determine and explain if the reference article supports the statement.  
    
Specifically, choose a label from "Fully substantiate", "Partially substantiate", and "Unsubstantiate". 
Further explanations of the labels are as follows: 
"Fully substantiated": The reference article fully substantiates the relevant part of the statement from the present article. 
"Partially substantiated": According to the reference article, there is a minor error in the statement but the error does not invalidate the purpose of the statement. 
"Unsubstantiate": The reference part does not substantiate any part of the statement. This could be because the statement is contradictory to, unrelated to, or simply missing from the reference article.  
    
Format your answer in JSON with two elements: "label" and "explanation". 
Your explanation should be short and concise. 
    
# The citing article
Title: {title} 
Statement: {statement}
    
# The reference article 
Title: {reference_title} 
Abstract: {reference_abstract} 
Excerpts: \n\n{reference_excerpts}
"""

    return prompt

In [5]:
example_prompt = create_prompt(df.iloc[22])
print(example_prompt)

   
You are an experienced scientific writer and editor. 
You will be given a statement from an article that cites a reference article and information from the reference article. 
You will determine and explain if the reference article supports the statement.  
    
Specifically, choose a label from "Fully substantiate", "Partially substantiate", and "Unsubstantiate". 
Further explanations of the labels are as follows: 
"Fully substantiated": The reference article fully substantiates the relevant part of the statement from the present article. 
"Partially substantiated": According to the reference article, there is a minor error in the statement but the error does not invalidate the purpose of the statement. 
"Unsubstantiate": The reference part does not substantiate any part of the statement. This could be because the statement is contradictory to, unrelated to, or simply missing from the reference article.  
    
Format your answer in JSON with two elements: "label" and "explanation"